# Bioimpedance Analysis - COMSOL 2-Electrode Modeling

This notebook analyzes and visualizes bioimpedance data from COMSOL simulations comparing **normal** and **cancer** tissue.

## Contents
1. Data Loading & Preprocessing
2. Data Exploration
3. Impedance Analysis
4. Electric Potential Visualization
5. Signal Processing
6. Normal vs Cancer Comparison
7. Statistical Analysis

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from scipy import signal, stats
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 12

## 1. Data Loading & Preprocessing

In [ ]:
def parse_complex(s):
    """Parse COMSOL complex number format (e.g., '1.23+4.56i' or '1.23-4.56i' or scientific notation)"""
    s = str(s).strip()
    # Handle scientific notation with 'E'
    s = s.replace('E', 'e')
    
    # Remove the trailing 'i'
    if s.endswith('i'):
        s = s[:-1]
    
    # Find the split point between real and imaginary parts
    # Look for + or - that is not part of scientific notation (e.g., e-10)
    idx = -1
    for i in range(len(s) - 1, 0, -1):
        if s[i] in ['+', '-'] and s[i-1] not in ['e', 'E']:
            idx = i
            break
    
    if idx == -1:
        # Only real part
        return complex(float(s), 0)
    
    real_part = float(s[:idx])
    imag_part = float(s[idx:])
    
    return complex(real_part, imag_part)


def load_comsol_data(filepath):
    """Load COMSOL CSV export with metadata parsing"""
    # Read metadata
    metadata = {}
    header_lines = 0
    
    with open(filepath, 'r') as f:
        for line in f:
            if line.startswith('%'):
                header_lines += 1
                parts = line[1:].strip().split(',', 1)
                if len(parts) == 2:
                    metadata[parts[0].strip()] = parts[1].strip()
            else:
                break
    
    # Read column headers and data
    df = pd.read_csv(filepath, skiprows=header_lines-1, header=0)
    
    # Rename columns for easier access
    df.columns = ['x', 'y', 'z', 'impedance_raw', 'potential_raw']
    
    # Parse complex numbers
    df['impedance'] = df['impedance_raw'].apply(parse_complex)
    df['potential'] = df['potential_raw'].apply(parse_complex)
    
    # Extract magnitude and phase
    df['Z_magnitude'] = np.abs(df['impedance'])
    df['Z_phase'] = np.angle(df['impedance'], deg=True)
    df['Z_real'] = df['impedance'].apply(lambda x: x.real)
    df['Z_imag'] = df['impedance'].apply(lambda x: x.imag)
    
    df['V_magnitude'] = np.abs(df['potential'])
    df['V_phase'] = np.angle(df['potential'], deg=True)
    df['V_real'] = df['potential'].apply(lambda x: x.real)
    df['V_imag'] = df['potential'].apply(lambda x: x.imag)
    
    return df, metadata


# Load both datasets
print("Loading Normal tissue data...")
df_normal, meta_normal = load_comsol_data('Data/Normal.csv')

print("Loading Cancer tissue data...")
df_cancer, meta_cancer = load_comsol_data('Data/Cancer.csv')

print(f"\n✓ Normal tissue: {len(df_normal)} nodes loaded")
print(f"✓ Cancer tissue: {len(df_cancer)} nodes loaded")

In [ ]:
# Display metadata
print("=" * 50)
print("NORMAL TISSUE - COMSOL Model Metadata")
print("=" * 50)
for key, value in meta_normal.items():
    print(f"{key}: {value}")

print("\n" + "=" * 50)
print("CANCER TISSUE - COMSOL Model Metadata")
print("=" * 50)
for key, value in meta_cancer.items():
    print(f"{key}: {value}")

## 2. Data Exploration

In [ ]:
# Display sample data
print("Normal Tissue - Sample Data:")
display(df_normal[['x', 'y', 'z', 'Z_magnitude', 'Z_phase', 'V_magnitude', 'V_phase']].head(10))

print("\nCancer Tissue - Sample Data:")
display(df_cancer[['x', 'y', 'z', 'Z_magnitude', 'Z_phase', 'V_magnitude', 'V_phase']].head(10))

In [ ]:
# Statistical summary
print("\n" + "=" * 60)
print("STATISTICAL SUMMARY - NORMAL TISSUE")
print("=" * 60)
print(df_normal[['x', 'y', 'z', 'Z_magnitude', 'Z_phase', 'V_magnitude', 'V_phase']].describe())

print("\n" + "=" * 60)
print("STATISTICAL SUMMARY - CANCER TISSUE")
print("=" * 60)
print(df_cancer[['x', 'y', 'z', 'Z_magnitude', 'Z_phase', 'V_magnitude', 'V_phase']].describe())

In [ ]:
# Geometry bounds
print("\nGeometry Bounds:")
print(f"X range: [{df_normal['x'].min():.2f}, {df_normal['x'].max():.2f}]")
print(f"Y range: [{df_normal['y'].min():.2f}, {df_normal['y'].max():.2f}]")
print(f"Z range: [{df_normal['z'].min():.2f}, {df_normal['z'].max():.2f}]")

## 3. Impedance Analysis

In [ ]:
# Compare impedance values
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Impedance Magnitude Distribution
ax1 = axes[0, 0]
ax1.hist(df_normal['Z_magnitude'], bins=50, alpha=0.7, label='Normal', color='blue')
ax1.hist(df_cancer['Z_magnitude'], bins=50, alpha=0.7, label='Cancer', color='red')
ax1.set_xlabel('Impedance Magnitude (Ω)')
ax1.set_ylabel('Frequency')
ax1.set_title('Impedance Magnitude Distribution')
ax1.legend()
ax1.set_yscale('log')

# Impedance Phase Distribution
ax2 = axes[0, 1]
ax2.hist(df_normal['Z_phase'], bins=50, alpha=0.7, label='Normal', color='blue')
ax2.hist(df_cancer['Z_phase'], bins=50, alpha=0.7, label='Cancer', color='red')
ax2.set_xlabel('Impedance Phase (degrees)')
ax2.set_ylabel('Frequency')
ax2.set_title('Impedance Phase Distribution')
ax2.legend()

# Real vs Imaginary Impedance (Cole-Cole plot style)
ax3 = axes[1, 0]
ax3.scatter(df_normal['Z_real'].iloc[0], -df_normal['Z_imag'].iloc[0], 
           s=200, alpha=0.8, label='Normal', color='blue', marker='o')
ax3.scatter(df_cancer['Z_real'].iloc[0], -df_cancer['Z_imag'].iloc[0], 
           s=200, alpha=0.8, label='Cancer', color='red', marker='s')
ax3.set_xlabel('Real(Z) (Ω)')
ax3.set_ylabel('-Imag(Z) (Ω)')
ax3.set_title('Nyquist Plot (Cole-Cole)')
ax3.legend()
ax3.grid(True)

# Box plot comparison
ax4 = axes[1, 1]
data_box = [df_normal['Z_magnitude'].values, df_cancer['Z_magnitude'].values]
bp = ax4.boxplot(data_box, labels=['Normal', 'Cancer'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')
ax4.set_ylabel('Impedance Magnitude (Ω)')
ax4.set_title('Impedance Comparison')

plt.tight_layout()
plt.savefig('impedance_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nKey Impedance Values:")
print(f"Normal Tissue Z: {df_normal['Z_magnitude'].iloc[0]:.6f} Ω @ {df_normal['Z_phase'].iloc[0]:.4f}°")
print(f"Cancer Tissue Z: {df_cancer['Z_magnitude'].iloc[0]:.6f} Ω @ {df_cancer['Z_phase'].iloc[0]:.6f}°")
print(f"\nRatio (Normal/Cancer): {df_normal['Z_magnitude'].iloc[0]/df_cancer['Z_magnitude'].iloc[0]:.2f}x")

## 4. Electric Potential Visualization

In [ ]:
# 3D Scatter plot of Electric Potential
fig = plt.figure(figsize=(16, 6))

# Normal tissue
ax1 = fig.add_subplot(121, projection='3d')
scatter1 = ax1.scatter(df_normal['x'], df_normal['y'], df_normal['z'], 
                       c=df_normal['V_magnitude'], cmap='viridis', s=5, alpha=0.6)
ax1.set_xlabel('X (m)')
ax1.set_ylabel('Y (m)')
ax1.set_zlabel('Z (m)')
ax1.set_title('Normal Tissue - Electric Potential Magnitude')
plt.colorbar(scatter1, ax=ax1, label='|V| (V)', shrink=0.6)

# Cancer tissue
ax2 = fig.add_subplot(122, projection='3d')
scatter2 = ax2.scatter(df_cancer['x'], df_cancer['y'], df_cancer['z'], 
                       c=df_cancer['V_magnitude'], cmap='plasma', s=5, alpha=0.6)
ax2.set_xlabel('X (m)')
ax2.set_ylabel('Y (m)')
ax2.set_zlabel('Z (m)')
ax2.set_title('Cancer Tissue - Electric Potential Magnitude')
plt.colorbar(scatter2, ax=ax2, label='|V| (V)', shrink=0.6)

plt.tight_layout()
plt.savefig('3d_potential.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 2D Slice visualization at different Z levels
def plot_2d_slice(df, z_level, title, ax, cmap='viridis'):
    """Plot 2D slice at given z level"""
    # Find nearest z level
    z_unique = df['z'].unique()
    z_nearest = z_unique[np.argmin(np.abs(z_unique - z_level))]
    
    # Filter data for this z level (with tolerance)
    tolerance = 0.5
    mask = np.abs(df['z'] - z_nearest) < tolerance
    df_slice = df[mask]
    
    if len(df_slice) > 10:
        # Create grid
        xi = np.linspace(df_slice['x'].min(), df_slice['x'].max(), 100)
        yi = np.linspace(df_slice['y'].min(), df_slice['y'].max(), 100)
        xi, yi = np.meshgrid(xi, yi)
        
        # Interpolate
        zi = griddata((df_slice['x'], df_slice['y']), df_slice['V_magnitude'],
                      (xi, yi), method='linear')
        
        im = ax.contourf(xi, yi, zi, levels=50, cmap=cmap)
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_title(f'{title}\n(Z ≈ {z_nearest:.1f} m)')
        return im
    return None


# Plot slices at different z levels
z_levels = [-2, -4.5, -7]
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for i, z in enumerate(z_levels):
    im1 = plot_2d_slice(df_normal, z, f'Normal - V', axes[0, i], 'viridis')
    if im1:
        plt.colorbar(im1, ax=axes[0, i], label='|V| (V)')
    
    im2 = plot_2d_slice(df_cancer, z, f'Cancer - V', axes[1, i], 'plasma')
    if im2:
        plt.colorbar(im2, ax=axes[1, i], label='|V| (V)')

plt.suptitle('Electric Potential Distribution at Different Z Slices', fontsize=14)
plt.tight_layout()
plt.savefig('2d_slices.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Signal Processing

In [ ]:
# Extract potential along a line (y=50, z=-2) to analyze spatial variation
def extract_line_profile(df, y_val, z_val, tolerance=1.0):
    """Extract potential along a line at fixed y and z"""
    mask = (np.abs(df['y'] - y_val) < tolerance) & (np.abs(df['z'] - z_val) < tolerance)
    df_line = df[mask].sort_values('x')
    return df_line

# Use center line
y_center = (df_normal['y'].max() + df_normal['y'].min()) / 2
z_center = -2  # Near surface

line_normal = extract_line_profile(df_normal, y_center, z_center, tolerance=2)
line_cancer = extract_line_profile(df_cancer, y_center, z_center, tolerance=2)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Potential magnitude along line
ax1 = axes[0, 0]
ax1.plot(line_normal['x'], line_normal['V_magnitude']*1e6, 'b-o', label='Normal', markersize=3, alpha=0.7)
ax1.plot(line_cancer['x'], line_cancer['V_magnitude']*1e6, 'r-s', label='Cancer', markersize=3, alpha=0.7)
ax1.set_xlabel('X position (m)')
ax1.set_ylabel('Potential Magnitude (µV)')
ax1.set_title(f'Potential Profile (Y≈{y_center:.0f}m, Z≈{z_center}m)')
ax1.legend()
ax1.grid(True)

# Potential phase along line
ax2 = axes[0, 1]
ax2.plot(line_normal['x'], line_normal['V_phase'], 'b-o', label='Normal', markersize=3, alpha=0.7)
ax2.plot(line_cancer['x'], line_cancer['V_phase'], 'r-s', label='Cancer', markersize=3, alpha=0.7)
ax2.set_xlabel('X position (m)')
ax2.set_ylabel('Potential Phase (degrees)')
ax2.set_title('Phase Profile')
ax2.legend()
ax2.grid(True)

# Gradient (spatial derivative) - approximation of electric field
ax3 = axes[1, 0]
if len(line_normal) > 2:
    grad_normal = np.gradient(line_normal['V_magnitude'].values, line_normal['x'].values)
    ax3.plot(line_normal['x'].values[1:-1], grad_normal[1:-1]*1e6, 'b-', label='Normal', alpha=0.7)
if len(line_cancer) > 2:
    grad_cancer = np.gradient(line_cancer['V_magnitude'].values, line_cancer['x'].values)
    ax3.plot(line_cancer['x'].values[1:-1], grad_cancer[1:-1]*1e6, 'r-', label='Cancer', alpha=0.7)
ax3.set_xlabel('X position (m)')
ax3.set_ylabel('dV/dx (µV/m)')
ax3.set_title('Electric Field Magnitude (Spatial Gradient)')
ax3.legend()
ax3.grid(True)

# Ratio of potentials
ax4 = axes[1, 1]
# Merge on x coordinates (approximate)
merged = pd.merge_asof(line_normal.sort_values('x')[['x', 'V_magnitude']], 
                       line_cancer.sort_values('x')[['x', 'V_magnitude']], 
                       on='x', suffixes=('_normal', '_cancer'), tolerance=1)
merged = merged.dropna()
if len(merged) > 0:
    ratio = merged['V_magnitude_normal'] / merged['V_magnitude_cancer']
    ax4.plot(merged['x'], ratio, 'g-o', markersize=3)
    ax4.axhline(y=ratio.mean(), color='k', linestyle='--', label=f'Mean ratio: {ratio.mean():.2f}')
ax4.set_xlabel('X position (m)')
ax4.set_ylabel('V_normal / V_cancer')
ax4.set_title('Potential Ratio (Normal/Cancer)')
ax4.legend()
ax4.grid(True)

plt.tight_layout()
plt.savefig('signal_processing.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# FFT Analysis of spatial potential distribution
def spatial_fft_analysis(df, title):
    """Perform FFT on spatial potential data"""
    # Get potential magnitude sorted by position
    df_sorted = df.sort_values(['x', 'y', 'z'])
    v_signal = df_sorted['V_magnitude'].values
    
    # Remove DC component (mean)
    v_signal = v_signal - np.mean(v_signal)
    
    # FFT
    n = len(v_signal)
    fft_result = np.fft.fft(v_signal)
    fft_magnitude = np.abs(fft_result[:n//2])
    fft_freq = np.fft.fftfreq(n)[:n//2]
    
    return fft_freq, fft_magnitude

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# FFT of Normal tissue
freq_n, mag_n = spatial_fft_analysis(df_normal, 'Normal')
axes[0].semilogy(freq_n[1:], mag_n[1:], 'b-', alpha=0.7)
axes[0].set_xlabel('Spatial Frequency (1/node)')
axes[0].set_ylabel('Magnitude')
axes[0].set_title('Spatial FFT - Normal Tissue')
axes[0].grid(True)

# FFT of Cancer tissue  
freq_c, mag_c = spatial_fft_analysis(df_cancer, 'Cancer')
axes[1].semilogy(freq_c[1:], mag_c[1:], 'r-', alpha=0.7)
axes[1].set_xlabel('Spatial Frequency (1/node)')
axes[1].set_ylabel('Magnitude')
axes[1].set_title('Spatial FFT - Cancer Tissue')
axes[1].grid(True)

plt.tight_layout()
plt.savefig('fft_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Normal vs Cancer Comparison

In [ ]:
# Create comprehensive comparison figure
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. Impedance comparison (bar chart)
ax1 = axes[0, 0]
z_vals = [df_normal['Z_magnitude'].iloc[0], df_cancer['Z_magnitude'].iloc[0]]
colors = ['steelblue', 'indianred']
bars = ax1.bar(['Normal', 'Cancer'], z_vals, color=colors, edgecolor='black')
ax1.set_ylabel('Impedance (Ω)')
ax1.set_title('Impedance Magnitude Comparison')
for bar, val in zip(bars, z_vals):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, 
             f'{val:.4f}', ha='center', va='bottom', fontsize=10)

# 2. Potential magnitude distribution comparison
ax2 = axes[0, 1]
ax2.hist(df_normal['V_magnitude']*1e6, bins=50, alpha=0.6, label='Normal', color='blue', density=True)
ax2.hist(df_cancer['V_magnitude']*1e6, bins=50, alpha=0.6, label='Cancer', color='red', density=True)
ax2.set_xlabel('Potential Magnitude (µV)')
ax2.set_ylabel('Density')
ax2.set_title('Potential Distribution Comparison')
ax2.legend()

# 3. Complex impedance comparison
ax3 = axes[0, 2]
ax3.arrow(0, 0, df_normal['Z_real'].iloc[0], df_normal['Z_imag'].iloc[0], 
         head_width=0.002, head_length=0.001, fc='blue', ec='blue', label='Normal')
ax3.arrow(0, 0, df_cancer['Z_real'].iloc[0], df_cancer['Z_imag'].iloc[0], 
         head_width=0.0001, head_length=0.00005, fc='red', ec='red', label='Cancer')
ax3.set_xlabel('Real(Z) (Ω)')
ax3.set_ylabel('Imag(Z) (Ω)')
ax3.set_title('Complex Impedance Phasor')
ax3.legend()
ax3.grid(True)
ax3.axis('equal')

# 4. Mean potential by region
ax4 = axes[1, 0]
# Divide into regions based on position
x_mid = df_normal['x'].median()
y_mid = df_normal['y'].median()

regions = ['Q1 (x<mid,y<mid)', 'Q2 (x>mid,y<mid)', 'Q3 (x<mid,y>mid)', 'Q4 (x>mid,y>mid)']
normal_means = []
cancer_means = []

for xc, yc in [(0, 0), (1, 0), (0, 1), (1, 1)]:
    mask_n = ((df_normal['x'] > x_mid) == xc) & ((df_normal['y'] > y_mid) == yc)
    mask_c = ((df_cancer['x'] > x_mid) == xc) & ((df_cancer['y'] > y_mid) == yc)
    normal_means.append(df_normal[mask_n]['V_magnitude'].mean() * 1e6)
    cancer_means.append(df_cancer[mask_c]['V_magnitude'].mean() * 1e6)

x_pos = np.arange(len(regions))
width = 0.35
ax4.bar(x_pos - width/2, normal_means, width, label='Normal', color='steelblue')
ax4.bar(x_pos + width/2, cancer_means, width, label='Cancer', color='indianred')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(regions, rotation=45, ha='right')
ax4.set_ylabel('Mean Potential (µV)')
ax4.set_title('Regional Potential Comparison')
ax4.legend()

# 5. Scatter plot of potential values
ax5 = axes[1, 1]
# Sample for visualization
sample_idx = np.random.choice(len(df_normal), min(1000, len(df_normal)), replace=False)
ax5.scatter(df_normal.iloc[sample_idx]['V_real']*1e6, df_normal.iloc[sample_idx]['V_imag']*1e9, 
           alpha=0.3, label='Normal', c='blue', s=10)
ax5.scatter(df_cancer.iloc[sample_idx]['V_real']*1e6, df_cancer.iloc[sample_idx]['V_imag']*1e9, 
           alpha=0.3, label='Cancer', c='red', s=10)
ax5.set_xlabel('Real(V) (µV)')
ax5.set_ylabel('Imag(V) (nV)')
ax5.set_title('Complex Potential Distribution')
ax5.legend()

# 6. Summary statistics table
ax6 = axes[1, 2]
ax6.axis('off')
summary_data = [
    ['Metric', 'Normal', 'Cancer', 'Ratio'],
    ['|Z| (Ω)', f'{df_normal["Z_magnitude"].iloc[0]:.6f}', f'{df_cancer["Z_magnitude"].iloc[0]:.6f}', 
     f'{df_normal["Z_magnitude"].iloc[0]/df_cancer["Z_magnitude"].iloc[0]:.1f}x'],
    ['∠Z (°)', f'{df_normal["Z_phase"].iloc[0]:.4f}', f'{df_cancer["Z_phase"].iloc[0]:.6f}', '-'],
    ['Mean |V| (µV)', f'{df_normal["V_magnitude"].mean()*1e6:.4f}', f'{df_cancer["V_magnitude"].mean()*1e6:.6f}',
     f'{df_normal["V_magnitude"].mean()/df_cancer["V_magnitude"].mean():.1f}x'],
    ['Std |V| (µV)', f'{df_normal["V_magnitude"].std()*1e6:.4f}', f'{df_cancer["V_magnitude"].std()*1e6:.6f}', '-'],
]

table = ax6.table(cellText=summary_data, loc='center', cellLoc='center',
                  colWidths=[0.3, 0.25, 0.25, 0.2])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.5)

# Style header row
for j in range(4):
    table[(0, j)].set_facecolor('#4472C4')
    table[(0, j)].set_text_props(color='white', weight='bold')

ax6.set_title('Summary Statistics', pad=20)

plt.suptitle('Normal vs Cancer Tissue - Bioimpedance Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('comparison_summary.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Statistical Analysis

In [ ]:
# Statistical tests
print("=" * 60)
print("STATISTICAL ANALYSIS")
print("=" * 60)

# T-test for potential magnitude
t_stat, p_value = stats.ttest_ind(df_normal['V_magnitude'], df_cancer['V_magnitude'])
print(f"\n1. Two-sample t-test (Potential Magnitude):")
print(f"   t-statistic: {t_stat:.4f}")
print(f"   p-value: {p_value:.2e}")
print(f"   Significant difference: {'Yes' if p_value < 0.05 else 'No'} (α=0.05)")

# Mann-Whitney U test (non-parametric)
u_stat, p_value_mw = stats.mannwhitneyu(df_normal['V_magnitude'], df_cancer['V_magnitude'])
print(f"\n2. Mann-Whitney U test (Potential Magnitude):")
print(f"   U-statistic: {u_stat:.4f}")
print(f"   p-value: {p_value_mw:.2e}")
print(f"   Significant difference: {'Yes' if p_value_mw < 0.05 else 'No'} (α=0.05)")

# Effect size (Cohen's d)
mean_diff = df_normal['V_magnitude'].mean() - df_cancer['V_magnitude'].mean()
pooled_std = np.sqrt((df_normal['V_magnitude'].std()**2 + df_cancer['V_magnitude'].std()**2) / 2)
cohens_d = mean_diff / pooled_std
print(f"\n3. Effect Size (Cohen's d): {cohens_d:.4f}")
if abs(cohens_d) < 0.2:
    effect_interpretation = "negligible"
elif abs(cohens_d) < 0.5:
    effect_interpretation = "small"
elif abs(cohens_d) < 0.8:
    effect_interpretation = "medium"
else:
    effect_interpretation = "large"
print(f"   Interpretation: {effect_interpretation} effect")

# Correlation analysis
print(f"\n4. Correlation Analysis:")
corr_normal = np.corrcoef(df_normal['V_magnitude'], df_normal['Z_magnitude'])[0, 1]
corr_cancer = np.corrcoef(df_cancer['V_magnitude'], df_cancer['Z_magnitude'])[0, 1]
print(f"   Normal: Corr(|V|, |Z|) = {corr_normal:.4f}")
print(f"   Cancer: Corr(|V|, |Z|) = {corr_cancer:.4f}")

In [ ]:
# Correlation heatmaps
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Normal tissue correlations
cols = ['x', 'y', 'z', 'Z_magnitude', 'V_magnitude', 'V_phase']
corr_normal = df_normal[cols].corr()
sns.heatmap(corr_normal, annot=True, fmt='.2f', cmap='coolwarm', ax=axes[0], 
            center=0, vmin=-1, vmax=1)
axes[0].set_title('Normal Tissue - Correlation Matrix')

# Cancer tissue correlations  
corr_cancer = df_cancer[cols].corr()
sns.heatmap(corr_cancer, annot=True, fmt='.2f', cmap='coolwarm', ax=axes[1],
            center=0, vmin=-1, vmax=1)
axes[1].set_title('Cancer Tissue - Correlation Matrix')

plt.tight_layout()
plt.savefig('correlation_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Key Findings Summary

In [ ]:
print("=" * 70)
print("                     KEY FINDINGS SUMMARY                          ")
print("=" * 70)

z_ratio = df_normal['Z_magnitude'].iloc[0] / df_cancer['Z_magnitude'].iloc[0]
v_ratio = df_normal['V_magnitude'].mean() / df_cancer['V_magnitude'].mean()

print(f"""
📊 IMPEDANCE ANALYSIS (at 50 kHz)
   ├─ Normal Tissue:  |Z| = {df_normal['Z_magnitude'].iloc[0]:.6f} Ω
   ├─ Cancer Tissue:  |Z| = {df_cancer['Z_magnitude'].iloc[0]:.6f} Ω
   └─ Ratio (Normal/Cancer): {z_ratio:.1f}x

⚡ ELECTRIC POTENTIAL
   ├─ Normal Mean |V|: {df_normal['V_magnitude'].mean()*1e6:.4f} µV
   ├─ Cancer Mean |V|: {df_cancer['V_magnitude'].mean()*1e6:.6f} µV  
   └─ Ratio (Normal/Cancer): {v_ratio:.1f}x

📈 STATISTICAL SIGNIFICANCE
   ├─ t-test p-value: {p_value:.2e}
   ├─ Mann-Whitney p-value: {p_value_mw:.2e}
   └─ Cohen's d effect size: {cohens_d:.2f} ({effect_interpretation})

🔬 INTERPRETATION
   Cancer tissue shows significantly LOWER impedance compared to normal
   tissue. This is consistent with known bioimpedance characteristics:
   - Cancer cells have altered membrane properties
   - Increased cellular water content
   - Changes in extracellular matrix
   - Higher ionic conductivity
""")

print("=" * 70)
print("Figures saved: impedance_analysis.png, 3d_potential.png,")
print("               2d_slices.png, signal_processing.png,")
print("               fft_analysis.png, comparison_summary.png,")
print("               correlation_analysis.png")
print("=" * 70)